<br> 

#### 0. Prepare dataset 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection, preprocessing

In [2]:
titanic_df = pd.read_csv("titanic_modified.csv")

titanic_target = titanic_df[['Survived']].copy()
titanic_data = titanic_df.copy()

del titanic_data['Survived']

In [3]:
train_data, test_data, train_label, test_label = model_selection.train_test_split(titanic_data, titanic_target,
                                                                                 test_size=0.3,
                                                                                 random_state=0)

In [4]:
enc = preprocessing.OneHotEncoder(categories='auto') 

train_label = enc.fit_transform(train_label).toarray()
test_label = enc.fit_transform(test_label).toarray()

<br> 

#### 1. Train the model

In [5]:
import tensorflow as tf
from tensorflow.keras import datasets, utils
from tensorflow.keras import models, layers, activations, initializers, losses, optimizers, metrics

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # https://stackoverflow.com/questions/35911252/disable-tensorflow-debugging-information

In [6]:
model = models.Sequential() 

model.add(layers.Dense(input_dim=8, units=256, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.Activation('elu')) # elu or relu

model.add(layers.Dense(units=512, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.Activation('elu')) 

model.add(layers.Dense(units=512, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.Activation('elu'))

model.add(layers.Dense(units=256, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.Activation('elu')) 
model.add(layers.Dropout(rate=0.5))

model.add(layers.Dense(units=2, activation='softmax')) # One-hot vector for 0 & 1

In [7]:
model.compile(optimizer=optimizers.Adam(), 
              loss=losses.categorical_crossentropy, 
              metrics=[metrics.categorical_accuracy]) 

In [8]:
# verbose의 값을 0으로 지정하면 silent mode로 fitting이 진행됩니다.

history = model.fit(train_data, train_label, batch_size=100, epochs=20, validation_split=0.3, verbose=0) 

<br> 

#### 2. Test the model before saving

In [9]:
result = model.evaluate(test_data, test_label)

print('loss (cross-entropy) :', result[0])
print('test accuracy :', result[1])

9/9 [==============================] - 0s 3ms/step - loss: 0.4461 - categorical_accuracy: 0.8358
loss (cross-entropy) : 0.4460698366165161
test accuracy : 0.8358209133148193


<br> 

#### 3. Save the trained model

In [10]:
# 저장되는 항목 
# - Model의 architecture
# - Model의 모든 weights (Parameter Theta)
# - The state of the optimizer (바로 모델 학습의 재개 가능)

model.save('trained_model.h5') # "Save" the model

# model.save_weights('trained_model.h5') # weights만 따로 저장도 가능함

<br> 

#### 4. Load the saved model 

In [11]:
model = models.load_model('trained_model.h5') # "Load" the "model"

# model.load_weights('trained_model.h5') # weights만 따로 불러올 수 있음

In [12]:
result = model.evaluate(test_data, test_label)

print('loss (cross-entropy) :', result[0])
print('test accuracy :', result[1])

9/9 [==============================] - 0s 3ms/step - loss: 0.4461 - categorical_accuracy: 0.8358
loss (cross-entropy) : 0.4460698366165161
test accuracy : 0.8358209133148193


<hr>


### Appendix) Save the model while training (+ Keras Callbacks API) 

<br>

#### Keras Callbacks API (Keras 콜백 함수)를 사용하는 이유 
<br>

- 모델의 **학습(fit)이 시작된 이후**에는 **학습 완료 전까지 사람이 컨트롤할 수 있는 것이 없음**
- 이를 해결하기 위한 도구가 Keras의 콜백 함수 (프로그래밍 전반에서의 콜백 함수 정의 : https://j.mp/3ibaAT4)
- ex) 학습 중 Learning rate 값을 변화시키기 / 학습 중 일정 시간 성능 개선이 없을 경우 학습 조기 종료 / 학습 중 모델 중간 저장 등
- Keras Callbacks API 공식 문서 : https://keras.io/api/callbacks/

<br>

#### Keras Callback examples
<br>

- **ModelCheckpoint** (학습 중간 저장) : https://deep-deep-deep.tistory.com/53 + 아래 코드 참고
- **EarlyStopping** (학습 조기 종료) : https://deep-deep-deep.tistory.com/55
- **ReduceLROnPlateau** (학습율 자동 조절) : https://deep-deep-deep.tistory.com/56 (Plateau란? @ https://j.mp/3B56FzJ)

In [ ]:
# 다양한 ModelCheckpoint 적용 예제 @ https://www.tensorflow.org/tutorials/keras/save_and_load?hl=ko#훈련하는_동안_체크포인트_저장하기
# 아래 Parameter들의 상세 설명 @ https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint

tf.keras.callbacks.ModelCheckpoint( 
    filepath, # 모델 저장 경로, if '{epoch:02d}-{val_loss:.5f}.h5' -> ex) 01(2자리 epoch 수)-0.39121(해당 epoch val_loss값).h5 
    monitor = 'val_loss', # 'loss', 'val_loss', 'accuracy', etc.
    verbose = 0, # 0 or 1
    save_best_only = False, # True : monitor 중인 지표 기준 가장 좋은 모델 저장 or False : 하단 save_freq 기준 주기적 저장
    save_weights_only = False, # True == model.save_weights(filepath) or False == model.save(filepath) 
    mode = 'auto', # 'auto', 'min', 'max'
    save_freq = 'epoch', # 'epoch' or integer(== # of batches) 
    save_freq = 5 * batch_size # == saves the model's weights every 5 epochs (variable 'batch_size' should be set already)
)

In [16]:
model = models.Sequential() 
model.add(layers.Dense(input_dim=8, units=256, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.Activation('elu')) # elu or relu
model.add(layers.Dense(units=512, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.Activation('elu')) 
model.add(layers.Dense(units=512, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.Activation('elu'))
model.add(layers.Dense(units=256, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.Activation('elu')) 
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(units=2, activation='softmax')) # One-hot vector for 0 & 1

model.compile(optimizer=optimizers.Adam(), 
              loss=losses.categorical_crossentropy, 
              metrics=[metrics.categorical_accuracy]) 

In [17]:
# 모델 체크포인트 파일(중간 저장 모델)을 저장할 경로 설정 
checkpoint_path = 'saved_models/titanic_4-layer_elu.h5' # 필히 saved_models 폴더를 먼저 만들어줘야 합니다

# 콜백함수 객체 생성
callback_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, 
                                                         monitor='val_loss', # default
                                                         save_best_only=True, # "Save" the "best" model "only"
                                                         verbose=0)

In [18]:
# 콜백함수를 호출하며 모델 학습 진행 
history = model.fit(train_data, train_label, 
                    batch_size=100, epochs=100, validation_split=0.3, verbose=0,
                    callbacks=[callback_checkpoint]) # 콜백 함수 추가

In [19]:
# 마지막 학습 완료 시점을 기준으로 한 모델 성능

result = model.evaluate(test_data, test_label)
print('loss (cross-entropy) :', result[0])
print('test accuracy :', result[1])

9/9 [==============================] - 0s 3ms/step - loss: 0.5024 - categorical_accuracy: 0.8284
loss (cross-entropy) : 0.502413809299469
test accuracy : 0.8283582329750061


In [20]:
# 가장 낮은 Validation loss를 기준으로 한 (저장된) 모델 성능

model = models.load_model('saved_models/titanic_4-layer_elu.h5') 

result = model.evaluate(test_data, test_label)
print('loss (cross-entropy) :', result[0])
print('test accuracy :', result[1])

9/9 [==============================] - 0s 3ms/step - loss: 0.4502 - categorical_accuracy: 0.8209
loss (cross-entropy) : 0.45016586780548096
test accuracy : 0.8208954930305481
